In [224]:
from augur.utils import json_to_tree
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from collections import Counter
import json
import requests

In [223]:
#putting these hex codes here to save them because I like them and don't have anything to plot right now
colors = {'almost_black':'#041219','dark_teal':'#025F73','teal':'#429194',
          'light_teal':'#A1D0BE','orange':'#BD6C28'}

In [272]:
def read_in_tree(animal):
    
    #Load tree
    tree_url = f'https://nextstrain.org/groups/blab/ncov/cross-species/{animal}'
    tree_json = requests.get(tree_url, headers={"accept":"application/json"}).json()
    
    #Put tree in Bio.Phylo format
    tree = json_to_tree(tree_json)
    
    return tree

In [39]:
def get_host_by_node(tree):
    #make dictionary mapping each node to it's inferred host
    host_by_node = {}

    for node in tree.find_clades():

        if hasattr(node, "node_attrs"):
            host = node.node_attrs["host"]["value"]
            host_by_node[node.name] = host
            
    return host_by_node

In [10]:
def get_parent(tree, child_clade):
    """
    Function that returns the path from root to specified clade
    """
    node_path = tree.get_path(child_clade)
    return node_path

In [44]:
def find_cross_species_transmissions(tree):
    
    host_by_node = get_host_by_node(tree)
    #keep track of nodes where cross-species transmissions occur
    #keep track of this in both directions
    human_to_animal = []
    animal_to_human = []

    for node in tree.find_clades():
        #get list of parents of this node
        parents = get_parent(tree, node)[0:-1]

        if len(parents)!=0:
            #get direct parent
            parent = parents[-1].name

            #find inferred host at node and direct parent
            node_host = host_by_node[node.name]
            parent_host = host_by_node[parent]

            #find cross-species transmissions
            if node_host!=parent_host:
                if node_host== 'Human':
                    animal_to_human.append(node.name)
                else:
                    human_to_animal.append(node.name)
                    
    return human_to_animal, animal_to_human

Print the number of cross-species transmissions for each type of animal

In [279]:
dog_tree = read_in_tree('dog')
human_to_dog, dog_to_human = find_cross_species_transmissions(dog_tree)
print(f'{len(human_to_dog)} human to DOG events\n{len(dog_to_human)} DOG to human events\n')

cat_tree = read_in_tree('cat')
human_to_cat, cat_to_human = find_cross_species_transmissions(cat_tree)
print(f'{len(human_to_cat)} human to CAT events\n{len(cat_to_human)} CAT to human events\n')

deer_tree = read_in_tree('deer')
human_to_deer, deer_to_human = find_cross_species_transmissions(deer_tree)
print(f'{len(human_to_deer)} human to DEER events\n{len(deer_to_human)} DEER to human events\n')

mink_tree = read_in_tree('mink')
human_to_mink, mink_to_human = find_cross_species_transmissions(mink_tree)
print(f'{len(human_to_mink)} human to MINK events\n{len(mink_to_human)} MINK to human events\n')

35 human to DOG events
15 DOG to human events

86 human to CAT events
30 CAT to human events

26 human to DEER events
16 DEER to human events

38 human to MINK events
4 MINK to human events



In [117]:
#find transmissions chains within the animal species after cross-species transmission events
def get_chains_after_species_switch(tree, human_to_animal):
    
    #keep track of all chains descending from each transmission event
    human_to_animal_chains = []
    
    #keep track of all nodes in the animal host
    all_animal_nodes = []
    
    #want all chains after the transmission event
    #so only need to trace transmission event to tips 
    for node in tree.find_clades(terminal=True):
        #only want the samples taken from animal hosts
        if hasattr(node, "node_attrs"):
            host = node.node_attrs["host"]["value"]
            if host!= 'Human':
                
                #find if this node was a (terminal) tranmission event
                if node.name in human_to_animal:
                    human_to_animal_chains.append([node.name])
                    all_animal_nodes.append(node.name)
                else:
                    #get list of parents of this node
                    parents = get_parent(tree, node)[0:-1]
                    parent_names = [x.name for x in parents]

                    #find where the cross-species event was
                    cross_species_parent = int([parent_names.index(x) for x in parent_names if x in human_to_animal][0])
                    #find chain from that parent to this terminal node
                    chain = parent_names[cross_species_parent:]+[node.name]
                    human_to_animal_chains.append(chain)
                    all_animal_nodes+=chain
                    
    all_animal_nodes = set(all_animal_nodes)

    return human_to_animal_chains, all_animal_nodes


In [218]:
def get_mutations_after_species_switch(tree, human_to_animal):
    #find all chains of transmission in non-human animal
    human_to_animal_chains, all_animal_nodes = get_chains_after_species_switch(tree, human_to_animal)
    
    #genes to aggregate mutations for
    genes = ['S', 'E', 'M', 'N', 'ORF1a', 'ORF1b', 'ORF3a', 'ORF6', 'ORF7a', 'ORF7b', 'ORF8', 'ORF9b']
    
    #dictionary to map node to mutations on that node
    muts_at_node = {}
    
    #keep track of all mutations observed on nodes in the non-human animal
    all_mutations_in_animal = {g:[] for g in genes}
    
    for node in tree.find_clades():
        if node.name in all_animal_nodes:
            mutations = node.branch_attrs['mutations']
            muts_at_node[node.name] = mutations
            for k,v in mutations.items():
                if k in genes:
                    all_mutations_in_animal[k].append(v)
    
    #get rid of entries for genes with no muts
    all_mutations_in_animal = {k:v for k,v in all_mutations_in_animal.items() if v}  
    #flatten lists of muts
    all_mutations_in_animal = {k:list(np.concatenate(v).flat) for k,v in all_mutations_in_animal.items()}
    #count occurrences of muts
    mutation_counts_in_animal = {k:Counter(v).most_common() for k,v in all_mutations_in_animal.items()}
    
    #make dataframe to give all cross-species transmission events, 
    #the transmission chains within that non-human animal, and the mutations that happen along those chains
    mutations_along_chains = []
    
    for chain in human_to_animal_chains:
        cross_species = chain[0]
        chain_len = len(chain)
        muts_on_chain = [muts_at_node[x] for x in chain]
        
        gene_muts_dict = {}
        for g in genes:
            gene_muts = list(np.concatenate([x[g] if g in x.keys() else [] for x in muts_on_chain]).flat)
            gene_col_title = f'{g}_muts'
            gene_muts_dict[gene_col_title] = gene_muts
            
        mutations_along_chains.append({'cross_species_event': cross_species, 
                                       'chain_len': chain_len, 'chain': chain, 
                                       'muts_on_chain': muts_on_chain, **gene_muts_dict})
        
    mutations_in_animal_chains = pd.DataFrame(mutations_along_chains)
    
    #find the number of sequences that descend from each cross-species transmission event
    mutations_in_animal_chains['num_seqs_after_event'] = mutations_in_animal_chains.groupby(['cross_species_event'])['cross_species_event'].transform('count')
    
    return mutations_in_animal_chains, mutation_counts_in_animal


In [280]:
mutations_in_dog, mutation_counts_in_dog = get_mutations_after_species_switch(dog_tree, human_to_dog)
mutations_in_cat, mutation_counts_in_cat = get_mutations_after_species_switch(cat_tree, human_to_cat)
mutations_in_deer, mutation_counts_in_deer = get_mutations_after_species_switch(deer_tree, human_to_deer)
mutations_in_mink, mutation_counts_in_mink = get_mutations_after_species_switch(mink_tree, human_to_mink)

In [275]:
mutation_counts_in_dog

{'S': [('Y655H', 2),
  ('K679N', 2),
  ('H681R', 2),
  ('L8V', 1),
  ('D867E', 1),
  ('A684V', 1),
  ('L5F', 1),
  ('T95I', 1),
  ('D253G', 1),
  ('A701V', 1),
  ('E484K', 1),
  ('V289I', 1),
  ('N950D', 1),
  ('V1228L', 1),
  ('S112L', 1),
  ('M1229L', 1),
  ('A520S', 1),
  ('F55-', 1),
  ('L1224F', 1),
  ('-142D', 1),
  ('-143V', 1),
  ('-144Y', 1),
  ('D145Y', 1),
  ('-211N', 1),
  ('I212L', 1),
  ('S255F', 1)],
 'E': [('P71S', 1)],
 'M': [('L87F', 1), ('G3D', 1), ('E19Q', 1), ('T63A', 1), ('I82T', 1)],
 'N': [('M234I', 2),
  ('P199L', 2),
  ('P67R', 1),
  ('S201G', 1),
  ('T379I', 1),
  ('P207S', 1),
  ('P67S', 1),
  ('P365L', 1),
  ('D377Y', 1),
  ('G34R', 1),
  ('R203K', 1),
  ('T148S', 1),
  ('S190I', 1),
  ('K373N', 1),
  ('M210I', 1),
  ('G18V', 1),
  ('R32H', 1),
  ('G179C', 1),
  ('K38R', 1),
  ('V270I', 1),
  ('P279Q', 1),
  ('A419V', 1),
  ('L13P', 1),
  ('-31E', 1),
  ('-32R', 1),
  ('-33S', 1),
  ('D63G', 1),
  ('N27S', 1)],
 'ORF1a': [('K141-', 2),
  ('S142-', 2),
  ('F

In [278]:
mutation_counts_in_cat

{'S': [('Q677H', 3),
  ('L452R', 2),
  ('T29I', 2),
  ('A520S', 2),
  ('L5F', 2),
  ('-144Y', 2),
  ('T95I', 2),
  ('L1063F', 1),
  ('E583D', 1),
  ('A626S', 1),
  ('T299A', 1),
  ('A771S', 1),
  ('A1078S', 1),
  ('T859I', 1),
  ('D1163Y', 1),
  ('T573I', 1),
  ('T716I', 1),
  ('E780Q', 1),
  ('H49Y', 1),
  ('L18F', 1),
  ('V1104L', 1),
  ('T572I', 1),
  ('D936N', 1),
  ('D138H', 1),
  ('H69-', 1),
  ('V70-', 1),
  ('-69H', 1),
  ('-70V', 1),
  ('D570A', 1),
  ('H1118D', 1),
  ('K558N', 1),
  ('P479T', 1),
  ('L938F', 1),
  ('S13I', 1),
  ('W152C', 1),
  ('E484K', 1),
  ('D950N', 1),
  ('Y144S', 1),
  ('R346K', 1),
  ('N501Y', 1),
  ('P681H', 1),
  ('Y145N', 1),
  ('M1229I', 1),
  ('G142S', 1),
  ('E180V', 1),
  ('A1174S', 1),
  ('T638R', 1),
  ('V1228L', 1),
  ('G1167V', 1),
  ('Y453F', 1),
  ('L141-', 1),
  ('G142-', 1),
  ('V143-', 1),
  ('Y144-', 1),
  ('L54F', 1),
  ('-156E', 1),
  ('-157F', 1),
  ('G158R', 1),
  ('V289I', 1),
  ('T250I', 1),
  ('S255F', 1),
  ('G142D', 1),
  ('F4

In [281]:
mutation_counts_in_deer

{'S': [('T19R', 4),
  ('Y144-', 3),
  ('A701V', 3),
  ('T478K', 2),
  ('P681R', 2),
  ('A263V', 2),
  ('D950N', 2),
  ('L141-', 2),
  ('G142-', 2),
  ('V143-', 2),
  ('G502C', 1),
  ('L48S', 1),
  ('Q613H', 1),
  ('M153I', 1),
  ('G142D', 1),
  ('E156-', 1),
  ('F157-', 1),
  ('R158G', 1),
  ('I882F', 1),
  ('T29I', 1),
  ('V701A', 1),
  ('V70L', 1),
  ('L1244I', 1),
  ('P1263Q', 1),
  ('V1228L', 1),
  ('G72R', 1),
  ('F140-', 1),
  ('D994Y', 1),
  ('H245Y', 1),
  ('T22I', 1),
  ('A27V', 1),
  ('G1085R', 1),
  ('T739I', 1),
  ('G446V', 1),
  ('V622I', 1),
  ('D80Y', 1),
  ('A522V', 1),
  ('I100M', 1),
  ('V1122L', 1),
  ('L18F', 1),
  ('S640C', 1),
  ('L922F', 1),
  ('G257S', 1),
  ('L5F', 1),
  ('T572I', 1),
  ('C136F', 1)],
 'E': [('A41V', 1)],
 'M': [('I82T', 4), ('R42M', 1), ('G78C', 1)],
 'N': [('R203M', 4),
  ('D377Y', 4),
  ('S194L', 3),
  ('L199P', 3),
  ('P199L', 2),
  ('M322I', 2),
  ('D371Y', 1),
  ('P67S', 1),
  ('A273V', 1),
  ('K342*', 1),
  ('P365L', 1),
  ('F314L', 1),


In [274]:
mutation_counts_in_mink

{'S': [('V143-', 22),
  ('L141-', 21),
  ('G142-', 21),
  ('Y144-', 15),
  ('F486L', 11),
  ('N501T', 11),
  ('Y453F', 9),
  ('G142D', 6),
  ('H69-', 4),
  ('V70-', 4),
  ('V367F', 3),
  ('L5F', 3),
  ('D1259Y', 3),
  ('S13I', 2),
  ('D1146H', 2),
  ('T95I', 2),
  ('Y144F', 2),
  ('V222A', 2),
  ('R452L', 2),
  ('A222V', 2),
  ('G261D', 1),
  ('L176F', 1),
  ('G832D', 1),
  ('G257D', 1),
  ('V6F', 1),
  ('A942S', 1),
  ('D950E', 1),
  ('L452M', 1),
  ('T1027I', 1),
  ('A575V', 1),
  ('T859I', 1),
  ('N360I', 1),
  ('M731I', 1),
  ('S305P', 1),
  ('P1069T', 1),
  ('T501N', 1),
  ('L241-', 1),
  ('L242-', 1),
  ('A243-', 1),
  ('I210-', 1),
  ('K113T', 1),
  ('E1182D', 1),
  ('E484G', 1),
  ('P209S', 1),
  ('K1205N', 1),
  ('M153I', 1),
  ('A262S', 1),
  ('G842D', 1),
  ('Q314K', 1),
  ('P9S', 1),
  ('V193L', 1),
  ('D574Y', 1),
  ('F18L', 1),
  ('T19R', 1),
  ('E156-', 1),
  ('F157-', 1),
  ('R158G', 1),
  ('L452R', 1),
  ('T478K', 1),
  ('P681R', 1),
  ('F486I', 1),
  ('G1223S', 1),
  

In [258]:
def get_mutation_frequencies(mutations_in_animal, mutation_counts_in_animal):
    #for each mutation that occurs 5 or more times independently in the nonhuman animal,
    #find the number of proportion of cross-over events where this mutation occurs
    #this will be slightly different than the counts in "mutation_counts_in_mink" because that 
    #counts all occurrences of the mutation (and mutation could have happened several times in parallel in same species jump clade)
    #*should i eventually weight this is some way by how big that nonhuman clade is?

    muts_occurring_more_than_3_times = {}
    for gene, muts in mutation_counts_in_animal.items():
        for mut,occurrences in muts:
            if occurrences>=3:
                if gene in muts_occurring_more_than_3_times.keys():
                    muts_occurring_more_than_3_times[gene].append(mut)
                else:
                    muts_occurring_more_than_3_times[gene] = [mut]


    cross_over_groups = mutations_in_animal.groupby('cross_species_event')

    total_num_human_to_animal_jumps = len(cross_over_groups)

    freq_individual_muts = {}

    for gene, muts in muts_occurring_more_than_3_times.items():
        for mut in muts:
            #count number of human to animal jumps where this mutation occurred
            mut_count = 0
            for x,c in cross_over_groups:
                mut_in_group = False
                for i,v in c.iterrows():
                    gene_header = f'{gene}_muts'

                    if mut in v[gene_header]:
                        mut_in_group = True

                if mut_in_group == True:
                    mut_count+=1
            #only muts that happened in 3 or more clades independently
            if mut_count>=3:        
                freq_individual_muts[f'{gene}:{mut}'] = mut_count/total_num_human_to_animal_jumps
            
            
    return freq_individual_muts


In [266]:
freq_individual_muts_mink = get_mutation_frequencies(mutations_in_mink, mutation_counts_in_mink)
freq_individual_muts_deer = get_mutation_frequencies(mutations_in_deer, mutation_counts_in_deer)
freq_individual_muts_dog = get_mutation_frequencies(mutations_in_dog, mutation_counts_in_dog)
freq_individual_muts_cat = get_mutation_frequencies(mutations_in_cat, mutation_counts_in_cat)

In [268]:
freq_individual_muts_mink

{'S:V143-': 0.24242424242424243,
 'S:L141-': 0.21212121212121213,
 'S:G142-': 0.21212121212121213,
 'S:Y144-': 0.15151515151515152,
 'S:F486L': 0.21212121212121213,
 'S:N501T': 0.21212121212121213,
 'S:Y453F': 0.21212121212121213,
 'S:G142D': 0.09090909090909091,
 'S:H69-': 0.12121212121212122,
 'S:V70-': 0.12121212121212122,
 'S:L5F': 0.09090909090909091,
 'S:A222V': 0.09090909090909091,
 'N:S194L': 0.09090909090909091,
 'ORF1a:G4177E': 0.3333333333333333,
 'ORF1a:L3606F': 0.2727272727272727,
 'ORF1a:L3829F': 0.09090909090909091,
 'ORF1a:S2083-': 0.09090909090909091,
 'ORF1a:L2084I': 0.09090909090909091,
 'ORF3a:T229I': 0.24242424242424243,
 'ORF3a:L219V': 0.24242424242424243,
 'ORF3a:H182Y': 0.18181818181818182,
 'ORF7a:S60-': 0.09090909090909091,
 'ORF7a:T61-': 0.09090909090909091,
 'ORF7a:Q62-': 0.09090909090909091,
 'ORF7a:F63-': 0.09090909090909091,
 'ORF7a:A64-': 0.09090909090909091,
 'ORF7a:R78-': 0.09090909090909091,
 'ORF7a:A79-': 0.09090909090909091,
 'ORF7a:R80-': 0.0909090